# Machine Learning - Assignment 1

## Naive Bayes Learning algorithm, Cross-validation, and ROC-Curves

The aim of the assignment is to implement:

* Naive Bayes learning algorithm for binary classification tasks
* Visualization to plot a ROC-curve
* A cross-validation test
* Visualization of the average ROC-curve of a cross-validation test

Follow the instructions and implement what is missing to complete the assignment. Some functions have been started to help you a little bit with the inputs or outputs of the function.

**Note:** You might need to go back and forth during your implementation of the code. The structure is set up to make implementation easier, but how you return values from the different functions might vary, and you might find yourself going back and change something to make it easier later on.

## Assignment preparations

We help you out with importing the libraries and reading the data.

Look at the output to get an idea of how the data is structured.

# Add Necessary Imports

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import e, pi, sqrt

# Read Data from CSV

In [122]:
data = pd.read_csv("./iris.csv").to_numpy()

# Load data to relevant variables

In [123]:
mapped, index, unique_arr = np.unique(data[:, -1], return_index=True, return_inverse=True)
print(f"Mapped({mapped})\nindex({index})\nunique_arr({unique_arr})")
print(f"Data Before : {data[:,-1]}")
data[:, -1] = unique_arr
print(f"Data After : {data[:,-1]}")

filtered_data_setVsVir = data[data[:, -1] != 1]
filtered_data_setVsVer = data[data[:, -1] != 2]
filtered_data_verVsVir = data[data[:, -1] != 0]
# print("Filtered Data : ", filtered_data)
# print(f"Test : {index[1:]} : {data}")
iris_setosa, iris_versicolor, iris_virginica = np.split(data, index[1:])
print(f"Iris Setosa : {len(iris_setosa)}\nIris Versicolor : {len(iris_versicolor)}\nIris Virginica : {len(iris_virginica)}")

Mapped(['Iris-setosa' 'Iris-versicolor' 'Iris-virginica'])
index([  0  50 100])
unique_arr([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2])
Data Before : ['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris

# Print trial values

In [124]:
print(f"Full data array (features and labels):\n{iris_setosa[:3]}\n")
print("###############\n")
print(f"Train features (first 4 columns):\n{iris_setosa[:3, :-1]}\n")
print(f"Labels (last column):\n{iris_setosa[:3, -1:]}\n")
print(f"Names of labels:\n{[[numb, name] for numb, name in enumerate(mapped)]}")

Full data array (features and labels):
[[5.1 3.5 1.4 0.2 0]
 [4.9 3.0 1.4 0.2 0]
 [4.7 3.2 1.3 0.2 0]]

###############

Train features (first 4 columns):
[[5.1 3.5 1.4 0.2]
 [4.9 3.0 1.4 0.2]
 [4.7 3.2 1.3 0.2]]

Labels (last column):
[[0]
 [0]
 [0]]

Names of labels:
[[0, 'Iris-setosa'], [1, 'Iris-versicolor'], [2, 'Iris-virginica']]


## Data handling functions

As a start, we are going to implement some basic data handling functions to use in the future.

### 1) Split class into a train and test set

First, we need to be able to split the class into a train and test set.

In [125]:
split_ratio=0.8
def train_test_split(class_data):

    class_data = np.array(class_data)
    
    # Extract features and labels
    features = class_data[:, :-1]
    labels = class_data[:, -1]
    # print(f"Train Test Split Labels : {labels}")
    
    unique_classes = np.unique(labels)
    
    train_data = []
    test_data = []
    
    for cls in unique_classes:
        cls_indices = np.where(labels == cls)[0]
        np.random.shuffle(cls_indices)
        split_index = int(len(cls_indices) * split_ratio)
        train_indices = cls_indices[:split_index]
        test_indices = cls_indices[split_index:]
        train_data.extend(class_data[train_indices])
        test_data.extend(class_data[test_indices])
    
    train_data = np.array(train_data)
    test_data = np.array(test_data)
    
    np.random.shuffle(train_data)
    np.random.shuffle(test_data)
    
    return train_data, test_data

In [126]:
# Test the train_test_split function
train_data_setVsVer, test_data_setVsVer = train_test_split(filtered_data_setVsVer)
train_data_setVsVir, test_data_setVsVir = train_test_split(filtered_data_setVsVir)
train_data_verVsVir, test_data_verVsVir = train_test_split(filtered_data_verVsVir)
# print(f"Test : {train_data[:,-1]}")

# Print the output
# print("Train Data : ",len(train_data_setVsVer))
# print("Test Data : ",len(test_data_setVsVer))

print("Train Data : ",train_data_setVsVer)
print("Test Data : ",test_data_setVsVer)

Train Data :  [[4.8 3.1 1.6 0.2 0]
 [5.1 3.8 1.6 0.2 0]
 [5.1 3.8 1.9 0.4 0]
 [5.9 3.2 4.8 1.8 1]
 [4.9 2.4 3.3 1.0 1]
 [5.8 2.7 3.9 1.2 1]
 [5.0 3.5 1.3 0.3 0]
 [5.8 2.7 4.1 1.0 1]
 [4.8 3.0 1.4 0.1 0]
 [5.7 2.9 4.2 1.3 1]
 [4.9 3.1 1.5 0.1 0]
 [5.7 2.8 4.1 1.3 1]
 [5.5 2.5 4.0 1.3 1]
 [6.4 2.9 4.3 1.3 1]
 [6.2 2.2 4.5 1.5 1]
 [5.0 3.6 1.4 0.2 0]
 [6.5 2.8 4.6 1.5 1]
 [5.0 3.3 1.4 0.2 0]
 [5.4 3.4 1.5 0.4 0]
 [5.4 3.4 1.7 0.2 0]
 [6.6 2.9 4.6 1.3 1]
 [5.6 2.7 4.2 1.3 1]
 [5.6 3.0 4.5 1.5 1]
 [5.8 2.6 4.0 1.2 1]
 [6.3 3.3 4.7 1.6 1]
 [5.4 3.9 1.3 0.4 0]
 [4.5 2.3 1.3 0.3 0]
 [5.5 2.4 3.7 1.0 1]
 [5.0 3.4 1.6 0.4 0]
 [5.7 3.8 1.7 0.3 0]
 [4.7 3.2 1.6 0.2 0]
 [5.2 4.1 1.5 0.1 0]
 [5.6 3.0 4.1 1.3 1]
 [4.6 3.2 1.4 0.2 0]
 [5.0 3.2 1.2 0.2 0]
 [4.9 3.0 1.4 0.2 0]
 [4.8 3.4 1.6 0.2 0]
 [6.1 2.9 4.7 1.4 1]
 [5.7 2.6 3.5 1.0 1]
 [5.2 3.5 1.5 0.2 0]
 [6.6 3.0 4.4 1.4 1]
 [5.7 2.8 4.5 1.3 1]
 [4.3 3.0 1.1 0.1 0]
 [5.6 2.5 3.9 1.1 1]
 [4.4 2.9 1.4 0.2 0]
 [5.5 4.2 1.4 0.2 0]
 [5.3 3.7 1.5 0.2 0]

### 2) Split data into features and labels

The data as shown above is not always the optimal shape. To help us keep track of things, we can split the data into its features and labels seperately.

Each class is 4 features and 1 label in the same array: 

* **[feature 1, feature 2, feature 3, feature 4, label]**

It would help us later to have the features and labels in seperate arrays in the form: 

* **[feature 1, feature 2, feature 3, feature 4]** and **[label]**

Here you are going to implement this functionallity.

In [127]:
def class_split(data):
    features = data[:, :-1].astype(float)
    labels = data[:, -1].astype(int)
    # print(f"Class Split Debug \nFeatures : {len(features)}\nLabels : \n{labels}")
    return features, labels

We should first test the "**class_split**" function on one of the classes above (iris_setosa, etc...) to make sure it works properly.

In [128]:
# Test the class splitting function
train_features_setVsVer, train_labels_setVsVer = class_split(train_data_setVsVer)
train_features_setVsVir, train_labels_setVsVir = class_split(train_data_setVsVir)
train_features_verVsVir, train_labels_verVsVir = class_split(train_data_verVsVir)

# Print the output
print("Train Data Features : ",train_features_verVsVir)
print("Train Data Labels : ",train_labels_verVsVir)

Train Data Features :  [[6.1 3.  4.9 1.8]
 [6.2 3.4 5.4 2.3]
 [6.7 3.1 4.7 1.5]
 [6.  2.7 5.1 1.6]
 [7.7 2.6 6.9 2.3]
 [6.4 2.9 4.3 1.3]
 [7.7 2.8 6.7 2. ]
 [4.9 2.4 3.3 1. ]
 [5.8 2.7 5.1 1.9]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 5.6 2.4]
 [5.8 2.7 3.9 1.2]
 [6.5 3.  5.5 1.8]
 [6.1 2.8 4.  1.3]
 [6.3 3.3 6.  2.5]
 [7.3 2.9 6.3 1.8]
 [5.5 2.4 3.7 1. ]
 [6.9 3.1 4.9 1.5]
 [6.3 3.3 4.7 1.6]
 [6.1 2.8 4.7 1.2]
 [5.6 2.7 4.2 1.3]
 [6.3 2.8 5.1 1.5]
 [6.1 2.6 5.6 1.4]
 [7.4 2.8 6.1 1.9]
 [6.7 3.  5.2 2.3]
 [6.7 3.  5.  1.7]
 [7.  3.2 4.7 1.4]
 [5.  2.  3.5 1. ]
 [5.7 3.  4.2 1.2]
 [6.1 3.  4.6 1.4]
 [5.7 2.8 4.1 1.3]
 [5.8 2.6 4.  1.2]
 [5.9 3.2 4.8 1.8]
 [5.7 2.5 5.  2. ]
 [6.  2.2 4.  1. ]
 [6.3 3.4 5.6 2.4]
 [6.4 3.2 5.3 2.3]
 [6.5 2.8 4.6 1.5]
 [6.3 2.9 5.6 1.8]
 [6.4 3.2 4.5 1.5]
 [5.6 2.8 4.9 2. ]
 [6.  3.4 4.5 1.6]
 [5.1 2.5 3.  1.1]
 [6.7 2.5 5.8 1.8]
 [7.2 3.6 6.1 2.5]
 [5.7 2.8 4.5 1.3]
 [6.3 2.5 4.9 1.5]
 [6.7 3.3 5.7 2.5]
 [6.6 3.  4.4 1.4]
 [6.2 2.2 4.5 1.5]
 [6.5 3.  5.8 2.2]
 [6.2 2.

We should also try to **1)** first split a class into a train and test set, **2)** split each of these two into features and abels. In total there should be 4 arrays (2 feature and 2 label arrays).

In [129]:
# Use the above data handling functions to make 2 feature arrays and 2 label arrays. 
train_features_setVsVer, train_labels_setVsVer = class_split(train_data_setVsVer)
test_features_setVsVer, test_labels_setVsVer = class_split(test_data_setVsVer)

train_features_setVsVir, train_labels_setVsVir = class_split(train_data_setVsVir)
test_features_setVsVir, test_labels_setVsVir = class_split(test_data_setVsVir)

train_features_verVsVir, train_labels_verVsVir = class_split(train_data_verVsVir)
test_features_verVsVir, test_labels_verVsVir = class_split(test_data_verVsVir)


# TODO: Print the output (you may use multiple print-statements if you wish)
print("Train Data Features : ",train_features_verVsVir)
print("Train Data Labels : ",train_labels_verVsVir)
print("Test Data Features : ",test_features_verVsVir)
print("Test Data Labels : ",test_labels_verVsVir)

Train Data Features :  [[6.1 3.  4.9 1.8]
 [6.2 3.4 5.4 2.3]
 [6.7 3.1 4.7 1.5]
 [6.  2.7 5.1 1.6]
 [7.7 2.6 6.9 2.3]
 [6.4 2.9 4.3 1.3]
 [7.7 2.8 6.7 2. ]
 [4.9 2.4 3.3 1. ]
 [5.8 2.7 5.1 1.9]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 5.6 2.4]
 [5.8 2.7 3.9 1.2]
 [6.5 3.  5.5 1.8]
 [6.1 2.8 4.  1.3]
 [6.3 3.3 6.  2.5]
 [7.3 2.9 6.3 1.8]
 [5.5 2.4 3.7 1. ]
 [6.9 3.1 4.9 1.5]
 [6.3 3.3 4.7 1.6]
 [6.1 2.8 4.7 1.2]
 [5.6 2.7 4.2 1.3]
 [6.3 2.8 5.1 1.5]
 [6.1 2.6 5.6 1.4]
 [7.4 2.8 6.1 1.9]
 [6.7 3.  5.2 2.3]
 [6.7 3.  5.  1.7]
 [7.  3.2 4.7 1.4]
 [5.  2.  3.5 1. ]
 [5.7 3.  4.2 1.2]
 [6.1 3.  4.6 1.4]
 [5.7 2.8 4.1 1.3]
 [5.8 2.6 4.  1.2]
 [5.9 3.2 4.8 1.8]
 [5.7 2.5 5.  2. ]
 [6.  2.2 4.  1. ]
 [6.3 3.4 5.6 2.4]
 [6.4 3.2 5.3 2.3]
 [6.5 2.8 4.6 1.5]
 [6.3 2.9 5.6 1.8]
 [6.4 3.2 4.5 1.5]
 [5.6 2.8 4.9 2. ]
 [6.  3.4 4.5 1.6]
 [5.1 2.5 3.  1.1]
 [6.7 2.5 5.8 1.8]
 [7.2 3.6 6.1 2.5]
 [5.7 2.8 4.5 1.3]
 [6.3 2.5 4.9 1.5]
 [6.7 3.3 5.7 2.5]
 [6.6 3.  4.4 1.4]
 [6.2 2.2 4.5 1.5]
 [6.5 3.  5.8 2.2]
 [6.2 2.

Think a bit before going to the next task, what can easily go wrong in the above code?

## Naive Bayes learning algorithm

When implementing the Navie Bayes learning algorithm, we can break it down into a few components.

We will implement these components one at a time.

### 3) Calculate feature statistics

First, we need to implement a function that returns feature statistics (means, standard deviation, priors) for a given set of feature data for a single class. This is the equivalent of "training" the naive bayes model.

**Note 1:** Each feature gets its own mean and standard deviation!

**Note 2:** The way you structure the functions (what is returned) shapes the remainder of the assignment.

In [130]:
def calculate_feature_statistics(feature_data, class_labels):
    stats = {}
    unique_labels = np.unique(class_labels)
    for label in unique_labels:
        class_data = feature_data[class_labels == label].astype(float)
        stats[label] = {
            'mean': np.mean(class_data, axis=0),
            'std': np.sqrt(np.var(class_data, axis=0)),
            'prior': len(class_data) / len(feature_data)
        }
    return stats

To make sure the function works, we should test it before proceding.

In [131]:
# Make sure to use our previous class splitting function.
feature_stats_temp_setVsVer = calculate_feature_statistics(train_data_setVsVer, train_labels_setVsVer)
feature_stats_temp_setVsVir = calculate_feature_statistics(train_data_setVsVir, train_labels_setVsVir)
feature_stats_temp_verVsVir = calculate_feature_statistics(train_data_verVsVir, train_labels_verVsVir)

print("Setosa Feature Statistics : ",feature_stats_temp_verVsVir[1]) 
# print("Setosa vs Virginica Feature Statistics : ",feature_stats_temp_setVsVir) 
print("Versicolor Feature Statistics : ",feature_stats_temp_verVsVir[1]) 

Setosa Feature Statistics :  {'mean': array([5.9575, 2.7775, 4.27  , 1.34  , 1.    ]), 'std': array([0.5113157 , 0.33278184, 0.48228622, 0.19849433, 0.        ]), 'prior': 0.5}
Versicolor Feature Statistics :  {'mean': array([5.9575, 2.7775, 4.27  , 1.34  , 1.    ]), 'std': array([0.5113157 , 0.33278184, 0.48228622, 0.19849433, 0.        ]), 'prior': 0.5}


### 4) Gaussian probability density function (Gaussian PDF)

Now we need to implement the gaussian probability density function to use for a single datapoint.

**Note:** Look at the imports in the first cell at the top, it has some math numbers for easy use here. 

In [132]:
def gaussian_probability_density_function(x, mean, std):
    x = np.array(x)
    mean = np.array(mean)
    std = np.array(std)
    exponent = np.exp(-((x - mean) ** 2 / (2 * std ** 2)))
    return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

## Prepare Naive Bayes for binary classification

### 6) Prepare the data for inference

Before we train and test the naive bayes for multiple classes, we should get our data in order.

Similar to how we did previously, we should now split two classes into a train and test set, you may choose which two classes freely.

In [133]:
predictions_setVsVer = []
true_labels_setVsVer = []

predictions_setVsVir = []
true_labels_setVsVir = []

predictions_verVsVir = []
true_labels_verVsVir = []

### 7) Class A vs Class B for binary classification

**Note:** You might need to go back and forth a bit in the following cells during your implementation of your code.

We have to get the probability from two sets of classes and compare the two probabilities in order to make a propper prediction.

Here we will implement two functions to make this possible. We seperate these functions to make the implementation of the ROC-curve easier later on.

**Function 1: naive_bayes_prediction** 
* A function that returns the probabilities for each class the model for a single datapoint.

**Function 2: probabilities_to_prediction**
* A function that takes in probabilities and returns a prediction.

In [134]:
def naive_bayes_prediction(feature_stats, data_point):
    probabilities = {}
    exes = []
    probs = []
    total_probability = 0
    
    for class_label, stats in feature_stats.items():
        log_likelihood = 0
        for i, feature_value in enumerate(data_point):
            mean = stats['mean'][i]
            std_dev = stats['std'][i]
            exes.append(feature_value)
            # print(f"Naive Bayes Test : mean({mean}) : std_dev({std_dev})")
            prob_den = gaussian_probability_density_function(feature_value, mean, std_dev)
            probs.append(prob_den)
            log_likelihood += np.log(gaussian_probability_density_function(feature_value, mean, std_dev))
        log_prior = np.log(stats['prior'])
        probabilities[class_label] = np.exp(log_likelihood + log_prior)
        total_probability += probabilities[class_label]
    
    # Normalize probabilities
    for class_label in probabilities:
        probabilities[class_label] /= total_probability
    
    return probabilities, exes, probs

In [135]:
def probabilities_to_prediction(probabilities):
    class_prediction = max(probabilities, key=probabilities.get)
    return class_prediction

In [136]:
labels_info = [[numb, name] for numb, name in enumerate(mapped)]
class_labels = [label[0] for label in labels_info]
print(f"Label Information : {labels_info}")
print(f"Class Labels : {class_labels}")

Label Information : [[0, 'Iris-setosa'], [1, 'Iris-versicolor'], [2, 'Iris-virginica']]
Class Labels : [0, 1, 2]


## ROC-curve

A ROC curve, or *Receiver Operating Characteristic curve*, is a graphical plot that illustrates the performance of a binary classifier such as our Naive Bayes model.

More info can be found in the course material and here: [https://en.wikipedia.org/wiki/Receiver_operating_characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)

Another good illustration by Google can be found here: [https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

Now that we have a prediction model, we would want to try it out and test it using a ROC-curve.

### 8) True Positive Rate (TPR) and False Positive Rate (FPR)

From our prediction function we get probabilities, and for prediction purposes we have just predicted the one with the highest probability.

To plot a ROC-curve, we need the TPR and FPR for the binary classification. We will implement this here.

**Note 1:** The threshold is is a value that goes from 0 to 1. 

**Note 2:** One of the two classes will be seen as "the positive class" (prediction over the threshold) and the other as "the negative class" (prediction under the threshold).

**Note 3:** The threshold stepsize will decide the size of the returned TPR/FPR list. A value of 0.1 will give 10 elements (0 to 1 in increments of 0.1)

In [137]:
# Stepsize demonstration
print("Python list:", [x/10 for x in range(0,10,1)])

# Stepsize demonstration with numpy:
print("Numpy linspace:", np.linspace(0,1,11))
print("Numpy linspace (no endpoint):", np.linspace(0,1,10,endpoint=False))

Python list: [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
Numpy linspace: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Numpy linspace (no endpoint): [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]


In [138]:
def TPR_and_FPR(prediction_probabilities, test_labels, threshold_stepsize = 0.1):
    # Define thresholds
    thresholds = np.linspace(0, 1, int(1 / threshold_stepsize) + 1)

    # Initialize TPR and FPR lists
    TPR, FPR = [], []

    # Ensure binary labels
    unique_labels = np.unique(test_labels)
    print(f"TPR_FPR Unique Classes Debug : {unique_labels} : {len(unique_labels)}")
    if len(unique_labels) != 2:
        raise ValueError("The labels must be binary (two unique classes).")
    positive_class = unique_labels[1]  # Higher value as positive class
    true_binary = np.array([1 if label == positive_class else 0 for label in test_labels])

    # Iterate through thresholds
    for threshold in thresholds:
        # Predicted positives based on the threshold
        predicted_positive = (np.array(prediction_probabilities) >= threshold).astype(int)

        # Confusion matrix components
        TP = np.sum((predicted_positive == 1) & (true_binary == 1))
        FP = np.sum((predicted_positive == 1) & (true_binary == 0))
        FN = np.sum((predicted_positive == 0) & (true_binary == 1))
        TN = np.sum((predicted_positive == 0) & (true_binary == 0))

        # Calculate TPR and FPR
        tprs = TP / (TP + FN) if (TP + FN) > 0 else 0
        fprs = FP / (FP + TN) if (FP + TN) > 0 else 0

        # Append results
        TPR.append(tprs)
        FPR.append(fprs)

    return np.array(TPR), np.array(FPR)

How does the values change if you change the threshold stepsize? 

How does the values change if you change the classes you compare?

### 9) Plot the TPR and FPR

To better see what is going on, we can plot the TPR and FPR. We can also calculate the Area Under the ROC Curve (AUC or AUROC) at the same time.

In [139]:
def plot_ROC(TPR, FPR, class_1, class_2):
    auc_score = np.trapz(sorted(TPR), sorted(FPR))
    plt.plot(FPR, TPR, color='b', label=f"AUC = {auc_score:.3f}")
    plt.plot([0, 1], [0, 1], linestyle='dashdot', color='r', label='Random Guess')
    class_1_title = "Iris"
    class_2_title = "Iris"
    if class_1==0:
        class_1_title = class_1_title+"-setosa"
    elif class_1==1:
        class_1_title = class_1_title+"-versicolor"
    else:
        class_1_title = class_1_title+"-virginica"

    if class_2==0:
        class_2_title = class_2_title+"-setosa"
    elif class_2==1:
        class_2_title = class_2_title+"-versicolor"
    else:
        class_2_title = class_2_title+"-virginica"
    
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"ROC Curve: {class_1_title} vs. {class_2_title}")
    plt.legend()
    plt.show()

## Cross-validation

The final task is to take everything you have implemented so far and apply it in a cross-validation loop.

**Note 1:** To better reflect a real scenarios, you should shuffle the data before doing cross-validation.

**Note 2:** When using cross-validation, the interesting thing is the mean performance (mean AUC, mean accuracy, mean ROC-curve).

**Note 3:** This part is a bit more free in terms of implementation, but make sure to use some of the previously implemented functions.

### 10) Cross-validation loop

In [140]:
def cross_validation(data, target, folds = 10, threshold_stepsize = 0.1):
    fold_size = len(data) // folds
    unique_classes = np.unique(target)
    print(f"Cross Validation Unique Classes Debug : {unique_classes} : {len(unique_classes)}")
    if len(unique_classes) != 2:
        raise ValueError("ROC analysis requires exactly two classes.")
    
    class_1, class_2 = unique_classes[1], unique_classes[0]
    # print(f"Target({target}) : Class 1({class_1}) and Class 2({class_2})")
    all_TPR, all_FPR = [], []

    def data_merge_and_split():
        class_1_data = []
        class_2_data = []
        for i in range(data):
            if target[i] == unique_classes[0]:
                class_1_data.append(data[i])
            else:
                class_2_data.append(data[i])
        return class_1_data, class_2_data
    
    def get_nth_fold_data(labels, n, fold_size = 5):
        class_1_data, class_2_data = data_merge_and_split()
        len_1 = len(class_1_data)
        len_2 = len(class_2_data)
        fold_lim_1 = round(len_1/fold_size)
        fold_lim_2 = round(len_2/fold_size)

        fold_start_1 = (n*fold_lim_1)
        fold_start_2 = (n*fold_lim_2)

        fold_end_1 = fold_start_1+fold_lim_1
        fold_end_2 = fold_start_2+fold_lim_2

        returnData = []
        returnLabels = []
        for index in range(fold_start_1, fold_end_1+1):            
            returnData.append(class_1_data[index])
            returnLabels.append(labels[0])
        for index in range(fold_start_2, fold_end_2+1):            
            returnData.append(class_2_data[index])
            returnLabels.append(labels[1])

        return np.array(returnData), np.array(returnLabels)

    
    for i in range(folds):
        # Create train-test splits
        start, end = i * fold_size, (i + 1) * fold_size
        test_data = data[start:end]
        test_labels = target[start:end]
        train_data = np.concatenate((data[:start], data[end:]), axis=0)
        train_labels = np.concatenate((target[:start], target[end:]), axis=0)

        data_new, labels_new = get_nth_fold_data(unique_classes, i, folds)
        
        # Train Naive Bayes and predict probabilities
        feature_stats = calculate_feature_statistics(train_data, train_labels)
        prediction_probabilities = []
        for x in test_data:
            prob_temp,t1,t2 = naive_bayes_prediction(feature_stats, x) 
            prediction_probabilities.append(prob_temp[class_1])
        
        # Calculate TPR and FPR
        TPR, FPR = TPR_and_FPR(prediction_probabilities, test_labels, threshold_stepsize)
        if i==0:
            print(f"Sample True Positive Rate : {TPR}")
            print(f"Sample False Positive Rate : {FPR}")
        all_TPR.append(TPR)
        all_FPR.append(FPR)
    
    # Average TPR and FPR across folds
    mean_TPR = np.mean(all_TPR, axis=0)
    mean_FPR = np.mean(all_FPR, axis=0)
    
    # Plot ROC
    plot_ROC(mean_TPR, mean_FPR, class_1, class_2)

### 11) 10-fold Cross-validation on all classes

Test the "cross_validation" function on all the classes against eachother using 10 folds.

* Iris-setosa vs Iris-versicolor
* Iris-setosa vs Iris-virginica
* Iris-versicolor vs Iris-virginica

### Implement and test cross-validation function on all classes.

In [141]:
filtered_data_setVsVer = data[data[:, -1] != 2]
filtered_data_setVsVir = data[data[:, -1] != 1]
filtered_data_verVsVir = data[data[:, -1] != 0]
# print(f"Filtered Data : {filtered_data}")
prediction_class_setVsVer =['0', '1']
prediction_class_setVsVir =['0', '2']
prediction_class_verVsVir =['1', '2']

# print(f"Train Features : {train_features}\nTrain Labels : {train_labels}")
feature_stats_setVsVer = calculate_feature_statistics(train_features_setVsVer, train_labels_setVsVer)
feature_stats_setVsVir = calculate_feature_statistics(train_features_setVsVir, train_labels_setVsVir)
feature_stats_verVsVir = calculate_feature_statistics(train_features_verVsVir, train_labels_verVsVir)

setVsVer_X = []
setVsVer_probDen = []
for test_sample in test_data_setVsVer:
    probabilities, exes, prob_dens = naive_bayes_prediction(feature_stats_setVsVer, test_sample[:-1])
    for item in exes:
        setVsVer_X.append(item)
    for item in prob_dens:
        setVsVer_probDen.append(item)
    predicted_class = probabilities_to_prediction(probabilities)
    predictions_setVsVer.append(predicted_class)
    true_labels_setVsVer.append(test_sample[-1])
print(f"Testing Gaussian Probability Density for Versicolor")
print(f"Test Data Points : {setVsVer_X}")
print(f"Probability Densities : {setVsVer_probDen}\n")

setVsVir_X = []
setVsVir_probDen = []
for test_sample in test_data_setVsVir:
    probabilities, exes, prob_dens = naive_bayes_prediction(feature_stats_setVsVir, test_sample[:-1])
    for item in exes:
        setVsVir_X.append(item)
    for item in prob_dens:
        setVsVir_probDen.append(item)
    predicted_class = probabilities_to_prediction(probabilities)
    predictions_setVsVir.append(predicted_class)
    true_labels_setVsVir.append(test_sample[-1])
print(f"Testing Gaussian Probability Density for Virginica")
print(f"Test Data Points : {setVsVir_X}")
print(f"Probability Densities : {setVsVir_probDen}\n")

verVsVir_X = []
verVsVir_probDen = []
for test_sample in test_data_verVsVir:
    probabilities, exes, prob_dens = naive_bayes_prediction(feature_stats_verVsVir, test_sample[:-1])
    for item in exes:
        verVsVir_X.append(item)
    for item in prob_dens:
        verVsVir_probDen.append(item)
    predicted_class = probabilities_to_prediction(probabilities)
    predictions_verVsVir.append(predicted_class)
    true_labels_verVsVir.append(test_sample[-1])
print(f"Testing Gaussian Probability Density for Setosa")
print(f"Test Data Points : {verVsVir_X}")
print(f"Probability Densities : {verVsVir_probDen}\n")

Testing Gaussian Probability Density for Versicolor
Test Data Points : [4.6, 3.1, 1.5, 0.2, 4.6, 3.1, 1.5, 0.2, 5.2, 3.4, 1.4, 0.2, 5.2, 3.4, 1.4, 0.2, 6.0, 2.2, 4.0, 1.0, 6.0, 2.2, 4.0, 1.0, 5.5, 3.5, 1.3, 0.2, 5.5, 3.5, 1.3, 0.2, 5.0, 2.0, 3.5, 1.0, 5.0, 2.0, 3.5, 1.0, 4.8, 3.4, 1.9, 0.2, 4.8, 3.4, 1.9, 0.2, 5.4, 3.0, 4.5, 1.5, 5.4, 3.0, 4.5, 1.5, 5.0, 3.0, 1.6, 0.2, 5.0, 3.0, 1.6, 0.2, 5.1, 3.8, 1.5, 0.3, 5.1, 3.8, 1.5, 0.3, 5.0, 3.5, 1.6, 0.6, 5.0, 3.5, 1.6, 0.6, 5.6, 2.9, 3.6, 1.3, 5.6, 2.9, 3.6, 1.3, 4.4, 3.2, 1.3, 0.2, 4.4, 3.2, 1.3, 0.2, 6.1, 3.0, 4.6, 1.4, 6.1, 3.0, 4.6, 1.4, 4.8, 3.0, 1.4, 0.3, 4.8, 3.0, 1.4, 0.3, 5.8, 4.0, 1.2, 0.2, 5.8, 4.0, 1.2, 0.2, 6.2, 2.9, 4.3, 1.3, 6.2, 2.9, 4.3, 1.3, 5.7, 3.0, 4.2, 1.2, 5.7, 3.0, 4.2, 1.2, 6.7, 3.0, 5.0, 1.7, 6.7, 3.0, 5.0, 1.7, 6.1, 2.8, 4.7, 1.2, 6.1, 2.8, 4.7, 1.2, 5.5, 2.4, 3.8, 1.1, 5.5, 2.4, 3.8, 1.1]
Probability Densities : [0.5803119568880684, 0.722286669691712, 2.3311980470136002, 3.622314283087063, 0.02463068137344643, 0.75

### Calculate accuracy

In [142]:
accuracy_setVsVer = sum(p == t for p, t in zip(predictions_setVsVer, true_labels_setVsVer)) / len(true_labels_setVsVer)
print("For Iris-Setosa vs Iris-Versicolor")
print(f"True labels: {true_labels_setVsVer}")
print(f"Predictions: {predictions_setVsVer}")
print(f"Accuracy: {accuracy_setVsVer * 100:.2f}%\n")

accuracy_setVsVir = sum(p == t for p, t in zip(predictions_setVsVir, true_labels_setVsVir)) / len(true_labels_setVsVir)
print("For Iris-Setosa vs Iris-Virginica")
print(f"True labels: {true_labels_setVsVir}")
print(f"Predictions: {predictions_setVsVir}")
print(f"Accuracy: {accuracy_setVsVir * 100:.2f}% \n")

accuracy_verVsVir = sum(p == t for p, t in zip(predictions_verVsVir, true_labels_verVsVir)) / len(true_labels_verVsVir)
print("For Iris-Versicolor vs Iris-Virginica")
print(f"True labels: {true_labels_verVsVir}")
print(f"Predictions: {predictions_verVsVir}")
print(f"Accuracy: {accuracy_verVsVir * 100:.2f}%\n")

For Iris-Setosa vs Iris-Versicolor
True labels: [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1]
Predictions: [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1]
Accuracy: 100.00%

For Iris-Setosa vs Iris-Virginica
True labels: [2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2]
Predictions: [2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2]
Accuracy: 100.00% 

For Iris-Versicolor vs Iris-Virginica
True labels: [1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1]
Predictions: [1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1]
Accuracy: 95.00%



In [143]:
target_setVsVer = filtered_data_setVsVer[:, -1]
data_setVsVer = filtered_data_setVsVer[:, :-1] 

target_setVsVir = filtered_data_setVsVir[:, -1]
data_setVsVir = filtered_data_setVsVir[:, :-1] 

target_verVsVir = filtered_data_verVsVir[:, -1]
data_verVsVir = filtered_data_verVsVir[:, :-1] 

### Evaluate using ROC

In [144]:
cross_validation(data_setVsVer, target_setVsVer, folds=10, threshold_stepsize=0.1)
cross_validation(data_setVsVir, target_setVsVir, folds=10, threshold_stepsize=0.1)
cross_validation(data_verVsVir, target_verVsVir, folds=10, threshold_stepsize=0.1)

Cross Validation Unique Classes Debug : [0 1] : 2
TPR_FPR Unique Classes Debug : [0] : 1


ValueError: The labels must be binary (two unique classes).

# Questions for examination:

In addition to completing the assignment with all its tasks, you should also prepare to answer the following questions:

1) Why is it called "naive bayes"?
<br>Ans : The Naive Bayes algorithm is an implementation of the traditional Bayes Theorem which was named after `Reverend Thomas Bayes` who studied Binomial Distributions and their probabilities and hence the model that implemented this algorithm was also named Naive Bayes after `Reverend Thomas Bayes`. The term `Naive`came into the name because of the independence assumtions made to reach the theorem which at that time was considered a 'Naive' approach.

2) What are some downsides of the naive bayes learning algorithm?
<br>Ans : The Naive Bayes algorithm has the following demerits to it : 
    - Conditional Independence Assumption : The assumptions about independence made in the algorithm doesn't always hold in real life situations
    - Zero Probability Problem : Any words encountered in the test data that aren't in the training data results in class probability value of zero

3) When using ROC-curves, what is the theoretical best and worst result you can get?
<br>Ans : Theoretically, the best ROC curve would have a curve score of 1.0. This is the case where the predictions exactly match with actual values for all tested values within and outside the testing data. The worst case would be a curve score of 0.5. This would be the equivalent of randomly guessing the outcome.

4) When using ROC-curves, in this assignment for example, is a higher threshold-stepsize always better?  
<br>Ans : No, a smaller threshold step size would provide a smoother ROC curve but it could possibly increase the computational time, but inversely, a larger threshold step size would reduce precision but can complete computations faster. This is usually dynamic and can vary based on the dataset used.

5) When using cross-validation and ROC-curves, why is it important to take the correct mean values? What could go wrong?
<br>Ans : Compared to numeric representations, ROC curves can represent the average performance of the model over each folds of cross validation in a more visually pleasing and understandable method and hence is helpful for a human analyst.
This can be offset by incorrect mean values since incorrect mean values can grossly misrepresent the true performance of the model, often leading to wrongly reached conclusions about the model's effectiveness.

# Finished!